In [ ]:
pip install kazoo

## ZooKeeperLock Class:
   - `__init__`: Initializes the class with ZooKeeper connection details, lock name, logger, and timeout.
   - `create_lock`: Creates a lock node in ZooKeeper.
   - `acquire`: Acquires the lock, optionally blocking until the lock is acquired.
   - `release`: Releases the lock.
   - `__del__`: Destructor to destroy the lock.
   - `destroy_lock`: Destroys the lock by stopping and closing the ZooKeeper client connection.

In [5]:
import logging
import os
import time
from kazoo.client import KazooClient
from kazoo.recipe.lock import Lock

class ZooKeeperLock:  # Initializes the ZooKeeperLock class.
    def __init__(self, hosts, lock_name, logger=None, timeout=1):
        self.hosts = hosts
        self.timeout = timeout
        self.zk_client = KazooClient(hosts=self.hosts, logger=logger, timeout=self.timeout)
        self.lock_name = lock_name
        self.lock_handle = None
        self.logger = logger or logging.getLogger(__name__)

    def create_lock(self):  # Creates a lock node in ZooKeeper.
        try:
            self.zk_client.start(timeout=self.timeout)
            lock_path = os.path.join("/", "locks", self.lock_name)
            self.lock_handle = Lock(self.zk_client, lock_path)
        except Exception as ex:
            self.logger.error("Failed to create lock: %s", ex)
            raise

    def acquire(self, blocking=True, timeout=None): # Acquires the lock.
        if self.lock_handle is None:
            return False
        try:
            return self.lock_handle.acquire(blocking=blocking, timeout=timeout)
        except Exception as ex:
            self.logger.error("Failed to acquire lock: %s", ex)
            return False

    def release(self):  # Releases the lock.
        if self.lock_handle is None:
            return False
        try:
            self.lock_handle.release()
            return True
        except Exception as ex:
            self.logger.error("Failed to release lock: %s", ex)
            return False

    def __del__(self):  # Destructor to destroy the lock.
        self.destroy_lock()

    def destroy_lock(self):  # Destroys the lock by stopping and closing the ZooKeeper client connection.
        if self.zk_client is not None:
            self.zk_client.stop()
            self.zk_client.close()


In [6]:
zookeeper_hosts = 'zookeeper1:2181,zookeeper2:2181,zookeeper3:2181'

## Simulated Work


1. **Lock Initialization**: A lock named "test" is initialized for the ZooKeeper cluster.
2. **Lock Operations**: It tries to create and acquire the lock. If the lock is successfully acquired, it proceeds; if not, it logs a failure message and exits.
3. **Simulated Work**: Upon successful lock acquisition, it logs this event and then simulates a task that runs for 10 seconds (counting from 1 to 10 with a one-second pause between counts).
4. **Release Lock**: After the task, it releases the lock and logs this event.
5. **Exception Handling**: The main function is wrapped in a try-except block to catch and log any exceptions that might occur during execution, which helps in debugging.
7. **Run Guard**: The `if __name__ == "__main__":` guard ensures that the code in the block runs only if the script is executed directly (not imported as a module). This is useful for scripts intended to be run as standalone programs.

In [ ]:
def main():
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    lock_name = "test"
    lock = ZooKeeperLock(zookeeper_hosts, lock_name, logger=logging.getLogger())
    lock.create_lock()
    ret = lock.acquire()
    if not ret:
        logging.info("Failed to acquire lock!")
        return

    logging.info("Lock acquired! Doing something! Sleeping for 10 seconds.")
    for i in range(1, 11):
        time.sleep(1)
        print(i)

    lock.release()
    logging.info("Lock released.")

if __name__ == "__main__":
    try:
        main()
    except Exception as ex:
        print("An exception occurred: {}".format(ex))
